# Using the Union Find decoder in qiskit_qec

This notebook will walk you through the process of creating a code, inserting errors and running the union find decoder on it using the qiskit_qec framework. 

The union find decoder was first proposed by Nicolas Delfosse and Naomi H. Nickerson in 2020 in *Almost linear time decoding algorithm for topological codes*. The core idea of the algorithm is to take a syndrome and "convert" it to an erasure. This erasure can be efficiently decoded by an erasure decoder (e.g. peeling decoder in $O(N)$ time). 

It works by growing clusters which are started around non-typical vertices in the syndrome graph by one half-edge per step in every direction. When two clusters meet, they merge and are considered one cluster.
A cluster grows when it is **odd**, meaning that it contains an odd number atypical nodes. 

The set of edges that is fully grown (meaning grown twice in an unweighted implementation) at the point where there are no more odd clusters, is considered to be the erasure and passed on to the erasure decoder. 

First we need to import all the relevant stuff:

In [7]:
import random

from qiskit_qec.analysis.faultenumerator import FaultEnumerator
from qiskit_qec.decoders import UnionFindDecoder
from qiskit_qec.circuits import SurfaceCodeCircuit, RepetitionCodeCircuit, ArcCircuit
from qiskit_qec.noise.paulinoisemodel import PauliNoiseModel

Now we create a code:

In [2]:
code = RepetitionCodeCircuit(d=8, T=8)

Now we define a decoder that runs on our code. This may take a while, as the the decoding graph is quite expensive to compute.

Note that we pass in the logical value our circuit is encoding, as the node generation for a given measurement outcome uses this to generate the last round of syndromes. In a real application this logical would be computed as the majority outcome.

In [4]:
decoder = UnionFindDecoder(code=code, logical="0")

Next we are going to define our noise model. This describes what the probabilities for different errors are after different operations. It will be used in the next step to generate the errors.

In [5]:
p = 0.05

noise_model = PauliNoiseModel()
noise_model.add_operation("cx", {"ix": 1, "xi": 1, "xx": 1})
noise_model.add_operation("id", {"x": 1})
noise_model.add_operation("reset", {"x": 1})
noise_model.add_operation("measure", {"x": 1})
noise_model.add_operation("x", {"x": 1, "y": 1, "z": 1})
noise_model.set_error_probability("cx", p)
noise_model.set_error_probability("x", p)
noise_model.set_error_probability("id", p)
noise_model.set_error_probability("reset", p)
noise_model.set_error_probability("measure", p)

Now we will use a very handy class: FaultEnumerator.

This class is responsible of adding single error events to our circuit, which our decoder will try to find and correct for.

We pass in the circuit in our code encoding a logical $0$, as this what we prepared our decoder for and the noise model we prepared earlier.

When using the fault enumerator class, we normally use it in a circumstance, where we want to iterate over all possible single error events, but in this demonstration we only want one, so we are going to pick a random entry.

In [10]:
fault_enumerator = FaultEnumerator(circ=code.circuit["0"], model=noise_model)

index, labels, error, outcome = random.choice(list(fault_enumerator.generate()))

We can now pass this outcome (which includes the syndrome measurement rounds) to our decoder for decoding, after converting it to a string:

In [11]:
outcome = "".join([str(x) for x in outcome])

corrected_outcome = decoder.process(outcome)

We get a list of integers describing the final state of our code qubits. We can now check what our logical outcome is, which is computed by looking at the parity of the logical qubits in the code.

In [14]:
logical_outcome = sum([corrected_outcome[index] for index in code.css_z_logical[0]]) % 2

assert logical_outcome == 0
print("Final outcome: ", logical_outcome)